In [1]:
pip install tensorflow


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install -U pycryptodome

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt

2024-04-02 13:38:15.770173: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-02 13:38:15.771664: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-02 13:38:15.799367: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-02 13:38:15.799396: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-02 13:38:15.800090: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [2]:
dataset = tf.keras.preprocessing.image_dataset_from_directory("Dataset_P", shuffle = True, image_size  = (256, 256), batch_size = 32)

Found 2152 files belonging to 3 classes.


In [3]:
class_names = dataset.class_names
class_names

['Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy']

In [4]:
dataset = dataset.shuffle(10000, seed = 12)

In [5]:
train_ds = dataset.take(54)
len(train_ds)

54

In [6]:
left_ds= dataset.skip(54)
len(left_ds)

14

In [7]:
validation_ds = left_ds.take(6)
len(validation_ds)

6

In [8]:
test_ds= left_ds.skip(6)
len(test_ds)

8

In [9]:
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size = tf.data.AUTOTUNE)
val_ds = validation_ds.cache().shuffle(1000).prefetch(buffer_size = tf.data.AUTOTUNE)
test_ds = test_ds.cache().shuffle(1000).prefetch(buffer_size = tf.data.AUTOTUNE)

In [10]:
resize_and_rescale = tf.keras.Sequential([
    layers.experimental.preprocessing.Resizing(256,256),
    layers.experimental.preprocessing.Rescaling(1.0/255),
])

In [11]:
data_augmentation = tf.keras.Sequential([
    layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    layers.experimental.preprocessing.RandomRotation(0.2),
])

In [12]:
potato_model = models.Sequential([
    resize_and_rescale,
    data_augmentation,
    
    layers.Conv2D(32, (3,3), activation ='relu', input_shape = (32,256,256,3), padding = "same"),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation ='relu',padding = "same"),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation ='relu',padding = "same"),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation ='relu',padding = "same"),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation ='relu',padding = "same"),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation ='relu',padding = "same"),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation ='relu',padding = "same"),
    layers.MaxPooling2D(2,2),
    
    layers.Flatten(),
    layers.Dense(64,activation = 'relu'),
    layers.Dense(3, activation = 'softmax')
])

potato_model.build(input_shape = (32,256,256,3))

In [13]:
potato_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (32, 256, 256, 3)         0         
                                                                 
 sequential_1 (Sequential)   (32, 256, 256, 3)         0         
                                                                 
 conv2d (Conv2D)             (32, 256, 256, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2  (32, 128, 128, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (32, 128, 128, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (32, 64, 64, 64)          0         
 g2D)                                                 

In [14]:
potato_model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [15]:
potato_model.fit(
    train_ds,
    epochs = 10,
    batch_size = 32,
    verbose = 1,
    validation_data= val_ds,
)

Epoch 1/10
54/54 [==============================] - 32s 557ms/step - loss: 0.8829 - accuracy: 0.4890 - val_loss: 0.7691 - val_accuracy: 0.5990
Epoch 2/10
54/54 [==============================] - 31s 573ms/step - loss: 0.6066 - accuracy: 0.7170 - val_loss: 0.5275 - val_accuracy: 0.7448
Epoch 3/10
54/54 [==============================] - 31s 572ms/step - loss: 0.3665 - accuracy: 0.8553 - val_loss: 0.2098 - val_accuracy: 0.9219
Epoch 4/10
54/54 [==============================] - 31s 574ms/step - loss: 0.4069 - accuracy: 0.8356 - val_loss: 0.3573 - val_accuracy: 0.8854
Epoch 5/10
54/54 [==============================] - 31s 569ms/step - loss: 0.2702 - accuracy: 0.8941 - val_loss: 0.2986 - val_accuracy: 0.8594
Epoch 6/10
54/54 [==============================] - 31s 572ms/step - loss: 0.2608 - accuracy: 0.8958 - val_loss: 0.3122 - val_accuracy: 0.8646
Epoch 7/10
54/54 [==============================] - 32s 595ms/step - loss: 0.1960 - accuracy: 0.9236 - val_loss: 0.0936 - val_accuracy: 0.9635

In [29]:
potato_model.evaluate(test_ds)

8/8 [==============================] - 1s 135ms/step - loss: 0.0720 - accuracy: 0.9844


[0.07197387516498566, 0.984375]

In [35]:
def predict(model, img):
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)
    
    predictions = model.predict(img_array)
    
    predicted_class = class_names[np.argmax(predictions[0])]
    confidence = round(100*(np.max(predictions[0])), 2)
    return predicted_class, confidence

In [18]:
potato_model.save(f"../Models/potato_f")

INFO:tensorflow:Assets written to: ../Models/potato_f/assets


INFO:tensorflow:Assets written to: ../Models/potato_f/assets


In [50]:
from PIL import Image
import numpy as np
image = Image.open("../Potato/Dataset_P/Potato___Late_blight/1cd053f6-0016-4680-a924-af15aecd7fb2___RS_LB 4414.JPG")
# image_array = np.array(image.resize((256, 256))) / 255.0
# image_batch = np.expand_dims(image_array, axis=0)

In [51]:
prediction = predict(potato_model,image)
print(prediction)
# potato_names = ['Early Blight', 'Late Blight', 'Healthy']
# print(prediction)
# predicted_class = potato_names[np.argmax(prediction[0])]
# print("1")
# print(predicted_class)

1/1 [==============================] - 0s 18ms/step
('Potato___Late_blight', 95.5)
